In [3]:
import music21 as m21
from simplemelodygen.extensions import MultiInstanceTrainableMarkovChainMelodyGenerator

## Load MIDI file

In [23]:
midi = m21.converter.parse('midi/kanada.mid')

notes = []

for element in midi.flat:
    if isinstance(element, m21.note.Note):
        duration = round(element.quarterLength * 4) / 4
        notes.append([str(element.pitch), duration])

for note in notes[:10]:
    print(note)

['E4', 1.25]
['F#4', 1.0]
['A4', 0.25]
['G4', 0.5]
['A4', 0.25]
['G4', 0.5]
['A4', 0.25]
['A4', 0.25]
['F#4', 1.25]
['E4', 2.5]


In [24]:
stream = m21.stream.Stream()

for i in range(len(notes)):
    note = m21.note.Note(notes[i][0], quarterLength=notes[i][1])
    stream.append(note)

stream.show('midi')

## State space

In [25]:
states = set()

for i in range(len(notes)):
    note = (notes[i][0], notes[i][1])
    states.add(note)

for state in list(states)[:10]:
    print(state)

('F#4', 1.0)
('F#4', 1.25)
('E5', 1.0)
('E5', 1.25)
('C#5', 0.75)
('C#5', 0.5)
('C#5', 0.25)
('E4', 1.25)
('E4', 1.0)
('E4', 1.75)


## Training data

In [26]:
training_data = []

for i in range(len(notes)):
    training_data.append(m21.note.Note(notes[i][0], quarterLength=notes[i][1]))

for note in training_data[:10]:
    print(note)

<music21.note.Note E>
<music21.note.Note F#>
<music21.note.Note A>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note A>
<music21.note.Note F#>
<music21.note.Note E>


## Training model

In [27]:
model = MultiInstanceTrainableMarkovChainMelodyGenerator(list(states))
model.train([training_data])

## Test

In [90]:
generated_melody = model.generate(32)[0]

In [91]:
total_duration = 0
cropped_melody = []

for note in generated_melody:
    total_duration += note[1]
    if total_duration >= 8:
        break
    cropped_melody.append(note)

if total_duration < 8:
    remaining_duration = 8 - total_duration
    cropped_melody[-1] = (cropped_melody[-1][0], cropped_melody[-1][1] + remaining_duration)

generated_melody = cropped_melody

In [92]:
for note in generated_melody[:10]:
    print(note)

('F#4', 1.25)
('E4', 2.5)
('D4', 0.25)
('E4', 0.5)
('D4', 1.25)
('C#4', 0.75)
('D4', 0.5)
('E4', 0.25)
('F#4', 0.5)


In [96]:
stream = m21.stream.Stream()

for i in range(len(generated_melody)):
    note = m21.note.Note(generated_melody[i][0], quarterLength=generated_melody[i][1])
    stream.append(note)

stream.show('midi')

In [99]:
previous_melody = generated_melody
extended_melody, generated_melody = model.generate(32, previous_sequence=generated_melody)

total_duration = 0
cropped_melody = []

for note in generated_melody:
    total_duration += note[1]
    if total_duration >= 8:
        break
    cropped_melody.append(note)

if total_duration < 8:
    remaining_duration = 8 - total_duration
    cropped_melody[-1] = (cropped_melody[-1][0], cropped_melody[-1][1] + remaining_duration)

extended_melody = previous_melody + cropped_melody

generated_melody = extended_melody

In [100]:
stream = m21.stream.Stream()

for i in range(len(generated_melody)):
    note = m21.note.Note(generated_melody[i][0], quarterLength=generated_melody[i][1])
    stream.append(note)

stream.show('midi')

In [101]:
stream.write('midi', fp='generated_melody.mid')

'generated_melody.mid'